# 深層学習ノートブック-8 回帰タスクにおけるBackpropagation
回帰タスクの場合の誤差逆伝播を計算し、スクラッチで実装したbackwardとautogradの結果が等しくなることを確認する。   
誤差逆伝播の各表式は変わらず、損失関数がsoftmax→MSEに変えれば良いだけ。  
回帰のタスクだが、autogradとの比較をするだけなので、ここではMNISTデータセットを用いる。

In [25]:
import torch
import torch.nn.functional as F  #pytorchの便利関数はFでimportすることが多い。
import numpy as np
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# python debugerをインポート
import pdb

## ■ backward（逆伝播）のスクラッチ実装結果
※現場ではこれらの関数は別ファイルでモジュール化して一元管理すること。

In [38]:
# 線形変換部分(パラメタ)の勾配の算出関数。Aはl-1層目の値で他はl層目の値。
def linear_backward(A, W, b, Z):
     # pytorch.tensorの.grad属性と区別するために.grad_として属性を定義している
     W.grad_ = Z.grad_.T @ A
     # bの値が一つ変わると、全データ分Lへ影響を及ぼすので、行方向(データ数の方向)にZ.grad_を足す。
     b.grad_ = torch.sum(Z.grad_, dim=0)
     # l-1層目のAはl層目の誤差（=Z.grad_）とl層目のWから計算できる。
     A.grad_ = Z.grad_ @ W


# ReLU関数
def relu_backward(Z, A):
     # 上式の"l番目の層における損失の誤差"における2行目と3行目を組み合わせてdL/dZを算出している
     # A.grad_はdL/dA, (Z > 0).float()はReLu関数の偏微分に相当する。
     Z.grad_ = A.grad_ * ( Z > 0 ).float()


# softmax関数と交差エントロピーの計算を同じ関数で実装する（pytorchでもこうなっている）
def softmax_cross_entropy(X, y_true):
    '''
    X: input tensor.行は各データ、列は各クラスを想定。
    y_true: tensor。One-Hot Encoding済みの正解ラベル。  
    '''
    max_val = X.max(dim=1, keepdim=True).values
    # 各要素のe^xを計算（これが分子になる）
    e_x = (X - max_val).exp()
    denominator = e_x.sum(dim=1, keepdim=True) + 1e-10
    softmax_out = e_x / denominator
    loss = - (y_true * torch.log(softmax_out + 1e-10)).sum() / y_true.shape[0]

    return loss, softmax_out


# tensorの線形結合を返す関数
def linear_comb(X, W, B):
    '''
    X, W, B: torch.tensor
    '''
    return X @ W.T + B


# ReLUの実装
def ReLU(Z):
    '''
    Z: torch.tensor
    '''
    # torch.whereによって要素ごとに条件が真・偽のときで別の値を返せる
    # 下記では0より大きい要素はzの値そのままで、0以下は0.になる。
    return torch.where(Z > 0 , Z, 0.)


In [36]:
# MSEと恒等関数の実装
def MSE_Identity_func(y_pred, y_true):
    return torch.sum( (y_pred - y_true)**2 ) / y_true.shape[0], y_pred

In [98]:
# 隠れ層１層の場合のMLP（スクラッチ）
def forward_and_backward(X, W_1, W_2, B_1, B_2, y):
    '''
    X: features
    W: List of weights(each edge)
    B: List of Bias(each Layer)
    '''

    #########################
    #### 順伝播（forward）####
    #########################

    # 入力層→隠れ層
    Z1 = linear_comb(X, W_1, B_1)
    Z1.retain_grad()

    # 活性化関数適用
    A1 = ReLU(Z1)
    A1.retain_grad()

    # 隠れ層→出力層
    Z2 = linear_comb(A1, W_2, B_2)
    Z2.retain_grad()

    # 最終出力。回帰なので恒等関数の結果(Z2そのまま)がA2になる。その後、MSEを計算。
    loss, A2 = MSE_Identity_func(Z2, y)


    #########################
    #### 逆伝播（backward）####
    #########################

    # 最終出力→出力層の出力のbackward。ここで各層の出力の勾配を求める。
    # ここでは出力層の活性化関数は恒等関数とし、
    # その出力にsoftmax_cross_entropyを適用した結果をモデルの最終出力として考えている。
    Z2.grad_ = 2 * (A2 - y) / X.shape[0]
    # 出力層→隠れ層のbackward
    linear_backward(A1, W_2, B_2, Z2)
    # 隠れ層の出力側のbackward
    relu_backward(Z1, A1)
    # 隠れ層の入力側のbackward
    linear_backward(X, W_1, B_1, Z1)
    
    return loss, Z1, A1, Z2, A2
    

※↑autogradの結果との比較はしないので、.retain_grad()は削除した

## ■ MLPの学習ループへ組み込み

## 前準備

In [109]:
# 変数定義
learning_rate = 0.03
loss_log = []  #学習時の損失記録用のリスト

# データロード
dataset = datasets.load_digits()
feature_names = dataset['feature_names']
X = torch.tensor(dataset['data'], dtype=torch.float32)
y_true = torch.tensor(dataset['target'])

# shape確認
print(f'shape of X: {X.shape}')
print(X[1])
print('==========================')
print(f'shape of y_train: {y_true.shape}')
print(y_true)

# 予期しないブロードキャスティングを防ぐために、yのRankはXやパラメタと合わせた方がよい。
y_true = y_true.reshape(-1, 1)

# 学習データと検証データを8:2に分ける
X_train, X_val, y_train, y_val = train_test_split(X, y_true, test_size=0.2, random_state=42)

print(f'shape of train data: X_train:{X_train.shape}, y_train:{y_train.shape}')
print(f'shape of validation data: X_val:{X_val.shape}, y_val:{y_val.shape}')


# 学習データ・検証データの標準化。検証データの標準化には学習データの平均、標準偏差を使用することに注意。  
X_mean = X_train.mean()
X_std = X_train.std()
X_train = (X_train - X_mean) / X_std
X_val = (X_val - X_mean) / X_std

# データ数、特徴量数、隠れ層のノード数、最終的な出力数（回帰では1列）を定義
m, nf = X_train.shape
nh = 30
n_out = 1

# 入力層→隠れ層の重みW、バイアス項bの初期化
W_1 = torch.randn((nh, nf)) * torch.sqrt(torch.tensor(2./nf))
W_1.requires_grad = True
B_1 = torch.zeros(size=(1, nh), requires_grad=True) # 1 x 出力

# 隠れ層→出力層の重みW、バイアス項bの初期化
W_2 = torch.randn((n_out, nh)) * torch.sqrt(torch.tensor(2./nf))
W_2.requires_grad = True
B_2 = torch.zeros(size=(1, n_out), requires_grad=True) # 1 x 出力


shape of X: torch.Size([1797, 64])
tensor([ 0.,  0.,  0., 12., 13.,  5.,  0.,  0.,  0.,  0.,  0., 11., 16.,  9.,
         0.,  0.,  0.,  0.,  3., 15., 16.,  6.,  0.,  0.,  0.,  7., 15., 16.,
        16.,  2.,  0.,  0.,  0.,  0.,  1., 16., 16.,  3.,  0.,  0.,  0.,  0.,
         1., 16., 16.,  6.,  0.,  0.,  0.,  0.,  1., 16., 16.,  6.,  0.,  0.,
         0.,  0.,  0., 11., 16., 10.,  0.,  0.])
shape of y_train: torch.Size([1797])
tensor([0, 1, 2,  ..., 8, 9, 8])
shape of train data: X_train:torch.Size([1437, 64]), y_train:torch.Size([1437, 1])
shape of validation data: X_val:torch.Size([360, 64]), y_val:torch.Size([360, 1])


## ※ブロードキャスティングの注意

回帰のため、yのone-hotエンコーディングが不要になったので、  
読み込んだy(Rank1のTensor)をそのまま計算に使用すると、逆伝播の計算(下記)でZ2.grad_のshapeがおかしくなり、  
行列積の計算が出来なくなってエラーになる。  
これは不用意なブロードキャスティングが原因。

```
    Z2.grad_ = 2 * (A2 - y) / X.shape[0]
    # 出力層→隠れ層のbackward
    linear_backward(A1, W_2, B_2, Z2)  
```  

試しに下記の通り計算すると、欲しいshapeは(1437, 1)であるはずが、(1437, 1437)になってしまう。

In [94]:
# y_trainを読み込んだままのshapeで計算してみる。
Z2_grad_temp = 2 * (torch.randn((1437, 1)) - y_train.reshape(-1)) / X_train.shape[0]
Z2_grad_temp.shape

torch.Size([1437, 1437])

In [95]:
# A2と同じRank2に合わせる
Z2_grad_temp = 2 * (torch.randn((1437, 1)) - y_train.reshape(-1, 1)) / X_train.shape[0]
Z2_grad_temp.shape

torch.Size([1437, 1])

上記のようになるのは、ブロードキャスティングの下記のルールによる。  
読み込んだ時のy_trainのshapeは(1437)なので、ブロードキャスティングの際にまず(1, 1437)になり、  
その後A2のshape(1437, 1)に合うように(1437, 1437)へy_trainがブロードキャスティングされてしまう。  
このような不意なブロードキャスティングを防ぐためにも計算前にあるべき形にshapeしておいた方がよい。  

* ブロードキャスティングのルール  
    * rank数が異なる場合，少ない方の配列のshapeの左側にサイズ1の次元を追加する(例: (2, 3) -> (1, 2, 3))  
    * shapeの右側から値(サイズ数)を比較し、数が一致するか、サイズが1であればブロードキャスティングが可能  
    * サイズ1の次元を大きい方の次元のサイズへ拡大する。この際、値はコピーされる。

In [110]:
loss, Z1, A1, Z2, A2 = forward_and_backward(X_train, W_1, W_2, B_1, B_2, y_train)
loss.backward()

In [111]:
# autogradの結果と比較
print(torch.allclose(Z1.grad_, Z1.grad))
print(torch.allclose(A1.grad_, A1.grad))
print(torch.allclose(W_1.grad_, W_1.grad))
print(torch.allclose(B_1.grad_, B_1.grad))
print(torch.allclose(Z2.grad_, Z2.grad))
print(torch.allclose(A2.grad_, A2.grad))
print(torch.allclose(W_2.grad_, W_2.grad))
print(torch.allclose(B_2.grad_, B_2.grad))

True
True
True
True
True
True
True
True


autogradの結果と等しいことが分かる。